# Slicing and Views

ironArray is meant to store large arrays, but in practice you only want to access single elements or small parts of them.  Here you will learn how to do that with the help of so called *views*.  Views are just references to the part of a larger array that is interesting.

Let's start by creating an array:

In [2]:
import numpy as np
import iarray as ia

dtshape = ia.DTShape([1000, 10000], np.float64)
arr = ia.arange(dtshape)
arr

<IArray (1000, 10000) np.float64>

Now suppose that we just want to access the first 2x2 square.  We just use the NumPy notation for doing this:

In [3]:
view = arr[0:2, 0:2]
view

<IArray (2, 2) np.float64>

as you see, we get another `IArray` instance, but a special one: a *view*.  You can always check whether an array is a view or not with the `is_view()` method:

In [4]:
view.is_view()

True

Of course, our initial array is not a view:

In [5]:
arr.is_view()

False

You can visualize the data in the view in the same way than a regular `IArray`, i.e. using `ia.iarray2numpy`.  However, there is a handy shortcut for doing the same thing:

In [6]:
view.data


array([[0.0000e+00, 1.0000e+00],
       [1.0000e+04, 1.0001e+04]])

In general, it is pretty common to use the `.data` accessor when you want to retrieve the actual data out of an `IArray`:

In [8]:
arr[0:4, 0:4].data

array([[0.0000e+00, 1.0000e+00, 2.0000e+00, 3.0000e+00],
       [1.0000e+04, 1.0001e+04, 1.0002e+04, 1.0003e+04],
       [2.0000e+04, 2.0001e+04, 2.0002e+04, 2.0003e+04],
       [3.0000e+04, 3.0001e+04, 3.0002e+04, 3.0003e+04]])

So, retrieving the interesting data out of your IArray is pretty easy.  And this applies also to arrays that are stored persistently on disk.  ironArray will use the information about the data you want and will read and decompress only the part that is necessary.  And due to the double partitioning and fast compression codecs, this is in general very fast (TODO: add some benchmarks at the end of this tutorial).

**Note:** IArray objects support most of the NumPy indexing syntax for getting slices with some exceptions:

1) IArray does not have support for strides.

2) It does not implement advanced indexing tricks.

At any rate, whenever you want to use this functionality, you can always get a NumPy array out of an IArray (or a view of it) and apply your desired indexing there.  Remember that ironArray is meant for handling very large arrays, so there is no shame in getting the interesting slice as a NumPy object and then do your work over it.